# Space Invaders

## Dependencies

In [3]:
!pip install tensorflow==2.9.1 gym keras-rl2 gym[atari]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ROM instructions: https://github.com/openai/atari-py#roms

In [4]:
!python -m atari_py.import_roms roms

copying space_invaders.bin from roms/Space Invaders.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/space_invaders.bin


## Exploration and baseline

In [5]:
import gym
import random
import numpy as np

In [6]:
env = gym.make("SpaceInvaders-v4")
print(env.observation_space.shape)

(210, 160, 3)


In [7]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [8]:
EPISODES = 100
scores = []
for episode in range(1, EPISODES + 1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action = random.choice(range(env.action_space.n))
        n_state, reward, done, info = env.step(action)
        score += reward
    
    scores.append(score)
    print(f"Episode {episode}: Reward == {score}")

avg = np.mean(scores)
print(f"Average reward: {avg}")
env.close()

Episode 1: Reward == 105.0
Episode 2: Reward == 110.0
Episode 3: Reward == 65.0
Episode 4: Reward == 210.0
Episode 5: Reward == 110.0
Episode 6: Reward == 210.0
Episode 7: Reward == 85.0
Episode 8: Reward == 50.0
Episode 9: Reward == 10.0
Episode 10: Reward == 365.0
Episode 11: Reward == 410.0
Episode 12: Reward == 210.0
Episode 13: Reward == 180.0
Episode 14: Reward == 150.0
Episode 15: Reward == 110.0
Episode 16: Reward == 105.0
Episode 17: Reward == 315.0
Episode 18: Reward == 75.0
Episode 19: Reward == 160.0
Episode 20: Reward == 125.0
Episode 21: Reward == 135.0
Episode 22: Reward == 225.0
Episode 23: Reward == 155.0
Episode 24: Reward == 220.0
Episode 25: Reward == 135.0
Episode 26: Reward == 90.0
Episode 27: Reward == 55.0
Episode 28: Reward == 185.0
Episode 29: Reward == 60.0
Episode 30: Reward == 240.0
Episode 31: Reward == 395.0
Episode 32: Reward == 135.0
Episode 33: Reward == 120.0
Episode 34: Reward == 80.0
Episode 35: Reward == 335.0
Episode 36: Reward == 240.0
Episode 37

So the baseline is around 150.

## Model

In [9]:
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D, Resizing, Rescaling, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.image import rgb_to_grayscale
from tensorflow.keras.layers import Layer
from tensorflow.keras.utils import register_keras_serializable

In [10]:
@register_keras_serializable("atari")
class GrayscaleLayer(Layer):
  def call(self, input):
    return rgb_to_grayscale(input)


In [20]:
def build_model(window_size, height, width, channels, actions):
    model = Sequential(name="seqmodel")
    model.add(Input(shape=(window_size, height, width, channels)))
    model.add(Reshape((window_size * height, width, channels)))
    model.add(GrayscaleLayer(name="grayscale"))
    model.add(Resizing((window_size * height) // 2, width // 2))
    model.add(Rescaling(1./255)) # normalize to [0, 1]
    model.add(Reshape((window_size, height // 2, width // 2, 1)))
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu'))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    # model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [21]:
WINDOW_SIZE = 4
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [22]:
model = build_model(WINDOW_SIZE, height, width, channels, actions)

In [23]:
model.summary()

Model: "seqmodel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 840, 160, 3)       0         
                                                                 
 grayscale (GrayscaleLayer)  (None, 840, 160, 1)       0         
                                                                 
 resizing_1 (Resizing)       (None, 420, 80, 1)        0         
                                                                 
 rescaling_1 (Rescaling)     (None, 420, 80, 1)        0         
                                                                 
 reshape_3 (Reshape)         (None, 4, 105, 80, 1)     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 4, 25, 19, 32)     2080      
                                                                 
 conv2d_4 (Conv2D)           (None, 4, 11, 8, 64)      328

## Agent

In [24]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [25]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(
        EpsGreedyQPolicy(), 
        attr='eps', 
        value_max=1.0, 
        value_min=0.1, 
        value_test=0.2, 
        nb_steps=10000
    )
    memory = SequentialMemory(
        limit=1000, 
        window_length=WINDOW_SIZE
    )
    dqn = DQNAgent(
        model=model, 
        memory=memory, 
        policy=policy,
        enable_dueling_network=True, 
        dueling_type='avg', 
        nb_actions=actions, 
        nb_steps_warmup=1000
    )
    return dqn

In [26]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## Train

In [27]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  578/10000: episode: 1, duration: 15.003s, episode steps: 578, steps per second:  39, episode reward: 120.000, mean reward:  0.208 [ 0.000, 30.000], mean action: 2.405 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


 1375/10000: episode: 2, duration: 549.922s, episode steps: 797, steps per second:   1, episode reward: 225.000, mean reward:  0.282 [ 0.000, 30.000], mean action: 2.449 [0.000, 5.000],  loss: 2.336038, mean_q: 0.546484, mean_eps: 0.893125
 1770/10000: episode: 3, duration: 566.094s, episode steps: 395, steps per second:   1, episode reward: 125.000, mean reward:  0.316 [ 0.000, 25.000], mean action: 2.600 [0.000, 5.000],  loss: 2.454093, mean_q: 1.040243, mean_eps: 0.858520
 2477/10000: episode: 4, duration: 1009.691s, episode steps: 707, steps per second:   1, episode reward: 120.000, mean reward:  0.170 [ 0.000, 25.000], mean action: 2.535 [0.000, 5.000],  loss: 1.785974, mean_q: 1.009570, mean_eps: 0.808930
 3351/10000: episode: 5, duration: 1257.164s, episode steps: 874, steps per second:   1, episode reward: 410.000, mean reward:  0.469 [ 0.000, 200.000], mean action: 2.651 [0.000, 5.000],  loss: 4.198885, mean_q: 0.808294, mean_eps: 0.737785
 3851/10000: episode: 6, duration: 71

In [28]:
scores = dqn.test(env, nb_episodes=20, visualize=False)
np.mean(scores.history["episode_reward"])

Testing for 20 episodes ...
Episode 1: reward: 135.000, steps: 818
Episode 2: reward: 140.000, steps: 795
Episode 3: reward: 150.000, steps: 815
Episode 4: reward: 80.000, steps: 399
Episode 5: reward: 145.000, steps: 817
Episode 6: reward: 105.000, steps: 403
Episode 7: reward: 150.000, steps: 791
Episode 8: reward: 105.000, steps: 668
Episode 9: reward: 320.000, steps: 776
Episode 10: reward: 125.000, steps: 777
Episode 11: reward: 180.000, steps: 901
Episode 12: reward: 175.000, steps: 788
Episode 13: reward: 125.000, steps: 802
Episode 14: reward: 140.000, steps: 803
Episode 15: reward: 145.000, steps: 801
Episode 16: reward: 165.000, steps: 869
Episode 17: reward: 75.000, steps: 715
Episode 18: reward: 160.000, steps: 776
Episode 19: reward: 155.000, steps: 839
Episode 20: reward: 80.000, steps: 405


142.75